## 14.1 파이썬 모듈의 개요

In [1]:
print(__name__)

## 14.2 간단한 두 개의 모듈 예시

In [4]:
# File run_me.py ----------------------------

import printstuff 		# printstuff.py 탑재
printstuff.print_this('thing') 	# 함수 호출

Print this thing.


In [5]:
# File run_me.py -----------------------------

import printstuff # printstuff.py 파일 탑재

# 다음 문장은 탑재된 변수를 사용한다.
print('%s and %s' % (printstuff.x, printstuff.y))

100 and 200


In [6]:
#File run_me.py ------------------------------

from foo_vars import z, print_z_val
import foo_vars

z = 100
print_z_val()

Value of z is 0


In [8]:
foo_vars.z = 100
print_z_val()

Value of z is 100


In [9]:
# File do_areas.py ----------------------------------

import poly      # Import the file poly.py

def main():
    r = float(input('Enter radius:'))
    print('Area of circle is', poly.get_circle_area(r))
    x = float(input('Enter side:'))
    print('Area of square is', poly.get_square_area(x))

main()

Enter radius:45
Area of circle is 6361.723800000001
Enter side:10
Area of square is 100.0


## 14.3 import 문의 변형

In [10]:
from poly import get_circle_area, get_square_area

def main():
    r = float(input('Enter radius:'))
    print('Area of circle is', get_circle_area(r))

    x = float(input('Enter side:'))
    print('Area of square is', get_square_area(x))

In [11]:
from poly import *

## 14.4 \_\_all\_\_ 기호 사용하기

In [3]:
# File run_me.py ------------------------------

from module2 import *

pr_nice('x', x)
pr_nice('y', y)

The value of x is 1000.
And z is 10.
The value of y is 500.
And z is 10.


In [4]:
# File listing for run_me.py ----------------------

from module2 import * 		# x, y, pr_nice 탑재
from module2 import z 		# z 탑재


print('z is %i, for heaven’s sake!' % z)

z is 10, for heaven’s sake!


In [6]:
# File run_me.py -----------------------------

from mod_a import *     # 실패할 것이다.

print(_a)
print(__b)

NameError: name '_a' is not defined

In [7]:
# File run_me.py -----------------------------

from mod_a import _a, __b 	# 이제 동작할 것이다.

print(_a)
print(__b)

10
100


## 14.6 메인 모듈과 \_\_main\_\_ 

In [4]:
# File run_me.py -----------------------------------
import mod_a

x = 1
y = 2

print('My name is %s.\n' % __name__)

print(dir())

My name is __main__.

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'mod_a', 'quit', 'x', 'y']


In [10]:
# File start_me_up.py -----------------------------

def call_me_first():
    print('Hi, there, Python!')

if __name__ == '__main__':
    call_me_first()

Hi, there, Python!


## 14.7	상호 탑재 문제 해결하기!

In [1]:
# File run_me.py -----------------------------------
import mod_a
mod_a.funcA(5)

3


AttributeError: module 'mod_a' has no attribute 'funcA'

## 14.8 RPN 예시: 2개의 모듈로 나누기

In [22]:
#File rpn.py --------------------------------------

import re
import operator
from rpn_io import *

# 심볼 테이블: 변수의 값이 저장된다.

sym_tab = { }

stack = [] 	# 값을 가지고 있을 스택

# 스캐너: 심볼 테이블에 저장될 변수 이름을 인식하기 위한 항목 추가
# 그리고 심볼 테이블에 넣은 값을 대입한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*)?\.\d+", lambda s, t:
        stack.append(float(t))),
    (r"-?\d+", lambda s, t: stack.append(int(t))),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", lambda s, t:
        stack.append(t)),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
    (r"[=]", lambda s, t: assign_op()),
    ])

def assign_op():
    '''대입 연산 함수: 이름과 값을 스택에서 꺼내서,
    심볼 테이블 엔트리를 만든다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op2) == str: 	# 소스는 다른 변수일 수도 있다!
        op2 = sym_tab[op2]
    sym_tab[op1] = op2

def bin_op(action):
    '''바이너리 연산 평가 함수: 피연산자가 변수 이름이면,
    평가하기 전에 심볼 테이블에서 찾아서
    해당 값으로 대체한다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    stack.append(action(op1, op2))

def main():
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return

    for a_line in a_list:
        a_line = a_line.strip()
        if a_line: 
            tokens, unknown = scanner.scan(a_line)
            if unknown:
                print('Unrecognized input:', unknown)
    print(str(stack[-1]))

main()

Enter RPN source: rpn_junk.txt
16


## 14.9 RPN 예시: I/O 지침 추가하기

In [31]:
#File rpn.py --------------------------------------

import re
import operator
from rpn_io import *

# 심볼 테이블: 변수의 값이 저장된다.

sym_tab = { }

stack = [] 	# 값을 보관하기 위한 스택

# 스캐너: 심볼 테이블에 저장될 변수 이름을 인식하기 위한 항목 추가
# 그리고 심볼 테이블에 넣은 값을 대입한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*)?\.\d+", lambda s, t:stack.append(float(t))),
    (r"-?\d+", lambda s, t: stack.append(int(t))),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", lambda s, t:stack.append(t)),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
    (r"[=]", lambda s, t: assign_op()),
    ])

def assign_op():
    '''대입 연산 함수: 이름과 값을 스택에서 꺼내서,
    심볼 테이블 엔트리를 만든다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op2) == str: 	# 소스는 다른 변수일 수도 있다!
        op2 = sym_tab[op2]
    sym_tab[op1] = op2

def bin_op(action):
    '''바이너리 연산 평가 함수: 피연산자가 변수 이름이면,
    평가하기 전에 심볼 테이블에서 찾아서
    해당 값으로 대체한다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    stack.append(action(op1, op2))

def main():
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return
    
    for a_line in a_list:
        a_line = a_line.strip()
        if a_line.startswith('PRINTS'):
            do_prints(a_line[7:])
        elif a_line.startswith('PRINTLN'):
            do_println(a_line[8:])
        elif a_line.startswith('PRINTVAR'):
            do_printvar(a_line[9:], sym_tab)
        elif a_line.startswith('INPUT'):
            do_input(a_line[6:], sym_tab)
        elif a_line: 
            tokens, unknown = scanner.scan(a_line)
            if unknown:
                print('Unrecognized input:', unknown)

main()

Enter RPN source: rpn_hyp.txt
Enter side 1: 30
Enter side 2: 40
Hypotenuse equals 50.0 

## 14.10 RPN 예시 추가 변경
### 14.10.2	0이 아니면 되돌아가는 기능 추가하기

In [4]:
#File rpn.py --------------------------------------

import re
import operator
from rpn_io import *

# 심볼 테이블: 변수의 값이 저장된다.

sym_tab = { } 		# (변수를 위한) 심볼 테이블
stack = [] 		    # 값을 보관하기 위한 스택
pc = -1 			# 프로그램 카운터 


# 스캐너: 심볼 테이블에 저장될 변수 이름을 인식하기 위한 항목 추가
# 그리고 심볼 테이블에 넣은 값을 대입한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*)?\.\d+", lambda s, t:stack.append(float(t))),
    (r"-?\d+", lambda s, t: stack.append(int(t))),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", lambda s, t:stack.append(t)),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[>]", lambda s, t: bin_op(operator.gt)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
    (r"[=]", lambda s, t: assign_op()),
    (r"[?]", lambda s, t: jnz_op()) 
    ])

def assign_op():
    '''대입 연산 함수: 이름과 값을 스택에서 꺼내서,
    심볼 테이블 엔트리를 만든다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op2) == str: 	# 소스는 다른 변수일 수도 있다!
        op2 = sym_tab[op2]
    sym_tab[op1] = op2

def bin_op(action):
    '''바이너리 연산 평가 함수: 피연산자가 변수 이름이면,
    평가하기 전에 심볼 테이블에서 찾아서
    해당 값으로 대체한다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    stack.append(action(op1, op2))
    
def jnz_op():
    global pc
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    if op1:
        pc = int(op2)-1

def main():
    global pc 
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return
    pc = -1 
    while True:
        pc += 1
        if pc >= len(a_list):
            break 
        a_line = a_list[pc]
        try: 
            a_line = a_line.strip()
            if a_line.startswith('PRINTS'):
                do_prints(a_line[7:])
            elif a_line.startswith('PRINTLN'):
                do_println(a_line[8:])
            elif a_line.startswith('PRINTVAR'):
                do_printvar(a_line[9:], sym_tab)
            elif a_line.startswith('INPUT'):
                do_input(a_line[6:], sym_tab)
            elif a_line:
                tokens, unknown = scanner.scan(a_line)
                if unknown:
                    print('Unrecognized input:', unknown)
                    break
        except KeyError as e:
            print('Unrecognized symbol', e.args[0],
                    'found in line', pc)
            print(a_list[pc])
            break 

main()

Enter RPN source: mystery.txt
Enter number of fibos to print: 10
1 2 3 5 8 13 21 34 55 89 

### 14.10.3	초과(>)와 임의 숫자 획득(!)

In [7]:
#File rpn.py --------------------------------------

import re
import operator
from rpn_io import *
from random import randint

# 심볼 테이블: 변수의 값이 저장된다.

sym_tab = { } 		# (변수를 위한) 심볼 테이블
stack = [] 		    # 값을 보관하기 위한 스택
pc = -1 			# 프로그램 카운터 


# 스캐너: 심볼 테이블에 저장될 변수 이름을 인식하기 위한 항목 추가
# 그리고 심볼 테이블에 넣은 값을 대입한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*)?\.\d+", lambda s, t: stack.append(float(t))),
    (r"-?\d+", lambda s, t: stack.append(int(t))),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", lambda s, t: stack.append(t)),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[>]", lambda s, t: bin_op(operator.gt)),
    (r"[!]", lambda s, t: bin_op(randint)), 
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
    (r"[=]", lambda s, t: assign_op()),
    (r"[?]", lambda s, t: jnz_op())
])

    
def jnz_op():
    global pc
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    if op1:
        pc = int(op2)-1
        
def assign_op():
    '''대입 연산 함수: 이름과 값을 스택에서 꺼내서,
    심볼 테이블 엔트리를 만든다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op2) == str: 	# 소스는 다른 변수일 수도 있다!
        op2 = sym_tab[op2]
    sym_tab[op1] = op2

def bin_op(action):
    '''바이너리 연산 평가 함수: 피연산자가 변수 이름이면,
    평가하기 전에 심볼 테이블에서 찾아서
    해당 값으로 대체한다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    stack.append(action(op1, op2))

def main():
    global pc 
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return
    pc = -1 
    while True:
        pc += 1
        if pc >= len(a_list):
            break 
        a_line = a_list[pc]
        try: 
            a_line = a_line.strip()
            if a_line.startswith('PRINTS'):
                do_prints(a_line[7:])
            elif a_line.startswith('PRINTLN'):
                do_println(a_line[8:])
            elif a_line.startswith('PRINTVAR'):
                do_printvar(a_line[9:], sym_tab)
            elif a_line.startswith('INPUT'):
                do_input(a_line[6:], sym_tab)
            elif a_line:
                tokens, unknown = scanner.scan(a_line)
                if unknown:
                    print('Unrecognized input:', unknown)
                    break
        except KeyError as e:
            print('Unrecognized symbol', e.args[0],
                    'found in line', pc)
            print(a_list[pc])
            break 

main()

Enter RPN source: rpn_game.txt
Enter your guess: 30
Too low! Try again. Enter your guess: 50
Too high! Try again. Enter your guess: 40
Too low! Try again. Enter your guess: 45
Too high! Try again. Enter your guess: 42
Too low! Try again. Enter your guess: 43
Congrats! You got it! Play again? (1 = yes, 0 = no): 0


## 14.11 RPN: 모든 소스 코드 모으기

In [9]:
#File rpn.py --------------------------------------

import re
import operator
from random import randint
from rpn_io import *

sym_tab = { } 	# (변수를 위한) 심볼 테이블
stack = [] 	# 값을 보관할 스택
pc = -1 		# 프로그램 카운터

# 스캐너: 심볼 테이블에 저장될 변수 이름을 인식하기 위한 항목 추가
# 그리고 심볼 테이블에 넣은 값을 대입한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*)?\.\d+", lambda s, t:
        stack.append(float(t))),
    (r"-?\d+", lambda s, t: stack.append(int(t))),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", lambda s, t:
        stack.append(t)),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[>]", lambda s, t: bin_op(operator.gt)),
    (r"[!]", lambda s, t: bin_op(randint)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
    (r"[=]", lambda s, t: assign_op()),
    (r"[?]", lambda s, t: jnz_op())
])

def jnz_op():
    ''' 0이 아니면 이동하는 연산.
    피연산자를 검증한 후, 첫 번째 op를 테스트한다.
    0이 아니면, 프로그램 카운터를 op2 – 1로 설정한다.
    '''
    global pc
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    if op1:
        pc = int(op2) - 1 # Convert op to int format.

def assign_op():
    '''대입 연산 함수: 이름과 값을 스택에서 꺼내서,
    심볼 테이블 엔트리를 만든다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op2) == str: # Source may be another var!
        op2 = sym_tab[op2]
    sym_tab[op1] = op2

def bin_op(action):
    ''' 바이너리 연산 평가 함수: 피연산자가 변수 이름이면,
    평가하기 전에 심볼 테이블에서 찾아서
    해당 값으로 대체한다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    stack.append(action(op1, op2))

def main():
    '''메인 함수
    프로그램을 주도하는 함수다. 파일을 열고  a_list안에
    연산자를 넣어서, a_list의 문자열을 하나씩 처리한다.
    '''
    global pc
    dir('__main__')
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return
    pc = -1
    while True:
        pc += 1
        if pc >= len(a_list):
            break
        a_line = a_list[pc]
        try:
            a_line = a_line.strip()
            if a_line.startswith('PRINTS'):
                do_prints(a_line[7:])
            elif a_line.startswith('PRINTLN'):
                do_println(a_line[8:])
            elif a_line.startswith('PRINTVAR'):
                do_printvar(a_line[9:], sym_tab)
            elif a_line.startswith('INPUT'):
                do_input(a_line[6:], sym_tab)
            elif a_line:
                tokens, unknown = scanner.scan(a_line)
                if unknown:
                    print('Unrecognized input:', unknown)
                    break
        except KeyError as e:
            print('Unrecognized symbol', e.args[0],
                   'found in line', pc)
            print(a_list[pc])
            break

main()

Enter RPN source: rpn_io.txt
File not found. Re-enter.
Enter RPN source: rpn_game.txt
Enter your guess: 50
Too high! Try again. Enter your guess: 25
Too high! Try again. Enter your guess: 13
Too low! Try again. Enter your guess: 18
Too high! Try again. Enter your guess: 15
Too low! Try again. Enter your guess: 16
Congrats! You got it! Play again? (1 = yes, 0 = no): 0
